<div style="font-size:18pt; padding-top:20px; text-align:center">СЕМИНАР. <b>Кросс-валидация</b></div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin.study@mail.ru)</span></div>

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Содержание</span>
    <ol>
        <li><a href="#1">Подходы к разбиению исходных данных</a></li>
        <li><a href="#2">Оценка качества модели</a>
        <li><a href="#3">Выбор модели с кросс-валидацией</a></li>
        <li><a href="#4">Источники</a>
        </li>
    </ol>
</div>

<p><b>Подключение библиотек</b></p>

In [ ]:
from sklearn import datasets
from scipy import stats
import numpy as np

from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn import linear_model
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:16pt; font-weight:bold">1. Подходы к разбиению исходных данных</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

### Отложенная выборка (Holdout)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x = np.array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
y = np.array([1, 1, 0, 0, 1, 0, 1, 1, 0, 0])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=0)

In [ ]:
x_train.shape, y_train.shape

In [ ]:
x_train, y_train

### Кросс-валидация K-Folds

In [ ]:
from sklearn.model_selection import cross_val_score, cross_validate, KFold

In [ ]:
x = np.array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
y = np.array([1,1,0,0,1,0,1,1,0,0])

kf = KFold(n_splits=3, random_state=0)
splits = kf.split(x, y)
for train_index, test_index in splits:
    print(train_index, test_index)

In [ ]:
kf = KFold(n_splits=3, shuffle=True, random_state=0)
splits = kf.split(x, y)
for train_index, test_index in splits:
    print(train_index, test_index)

In [ ]:
splits = kf.split(x, y)
for indx, (train_index, test_index) in enumerate(splits):
    print("\nSplit", indx+1)
    print("Train set (x and y)")
    print(x[train_index],  y[train_index])
    print("Test set (x and y)")
    print(x[test_index],  y[test_index])

In [ ]:
from sklearn.utils import shuffle

# TODO

### Кросс-валидация Leave-One-Out (LOO)

In [ ]:
from sklearn.model_selection import LeaveOneOut

x = np.array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
y = np.array([1,1,0,0,1,0,1,1,0,0])

loo = LeaveOneOut()
for train, test in loo.split(x):
    print("{}{}".format(train, test))

### Стратифицированная выборка

In [ ]:
# TODO

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:16pt; font-weight:bold">2. Оценка качества модели</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

In [ ]:
def regression_dataset(intercept=2, slope=0.3, n=100, start_x=4, length_x=8, mu=0, sigma=0.5):
    f = lambda x: intercept + slope*x
    x = stats.uniform.rvs(size=n, loc=start_x, scale=length_x)
    e = stats.norm.rvs(size=n, loc=mu, scale=sigma)
    y_true = f(x) + e
    return x.reshape(-1,1), y_true, f

# Генерация исходных данных
X, y_true, f = regression_dataset()

In [ ]:
# График
xx = np.linspace(4, 12, 100)
plt.title("Initial Data")
plt.scatter(X, y_true, color="green", label="observed")
plt.plot(xx, f(xx), "-", color="SteelBlue", label="true function", zorder=1)
plt.grid(True)
plt.xlabel("$x$")
plt.ylabel("$y_{true}$")
plt.legend()
plt.show()

### Holdout

In [ ]:
REPEATS = 10

mses = np.full(REPEATS, np.inf)

for i in range(REPEATS):

    # Разбиение данных
    X_train, X_test, y_train, y_test = train_test_split(X, y_true, 
        test_size=0.3, 
        random_state=i)

    # Обучение
    linear_model = LinearRegression()
    linear_model.fit(X_train, y_train)
    
    # Предсказания для тестового множества
    y_test__pred = linear_model.predict(X_test)

    # Ошибки на тестовом множестве
    mses[i] = mean_squared_error(y_test, y_test__pred)
    print("{}) Test MSE: {}".format(i+1, mses[i]))

print("\nДоверительный интервал MSE: {:0.3f} +/- {:0.3f}".format(mses.mean(), mses.std() * 2.0))

### K-Folds

In [ ]:
REPEATS = 10

mses = np.full(REPEATS, np.inf)

kf = KFold(n_splits=5, shuffle=True)

for i in range(REPEATS):
    
    kf.random_state = i
    
    # Обучение
    mse_list = - cross_val_score(linear_model, X, y_true, cv=kf, scoring="neg_mean_squared_error")
    
     # Средняя ошибка на тестовом множестве
    mses[i] = mse_list.mean()
    print("{}) Test MSE: {}".format(i+1,  mses[i]))
    
print("\nДоверительный интервал MSE: {:0.3f} +/- {:0.3f}".format(mses.mean(), mses.std() * 2.0))

<a name="3"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:16pt; font-weight:bold">3. Выбор модели с кросс-валидацией</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

In [ ]:
def regression_dataset():
    n = 100
    x = stats.uniform.rvs(size=n, loc=0, scale=5, random_state=10)
    f = lambda x:  np.sin(x)
    y_true = stats.norm.rvs(size=n, loc=0, scale=0.2, random_state=10) + f(x)
    return x.reshape(-1,1), y_true, f

# Генерация исходных данных
X, y_true, f = regression_dataset()

# График
xx = np.linspace(0, 5, 100)
plt.title("Initial Data")
plt.scatter(X, y_true, color="green", label="observed")
plt.plot(xx, f(xx), "-", color="SteelBlue", label="true function", zorder=1)
plt.grid(True)
plt.xlabel("$x$")
plt.ylabel("$y_{true}$")
plt.legend()
plt.show()

In [ ]:
SPLITS = 5
MAX_DEGREE = 9

#### Вариант 1

Замечание: Важно, чтобы сплиты `KFold` были одинаковыми для всех степеней. Поэтому далее используется `random_state`. Либо можно перетасовать данные перед кросс-валидацией и установить `shuffle` равным `False` для `KFold`

In [ ]:
# Разбиение данных на обучающее и тестовое множества
X_train, X_test, y_train, y_test = train_test_split(X, y_true, test_size=0.3, random_state=200)

# Инициализация делителя для кросс-валидации
# Замечание: нет необходимости использовать shuffle, т.к. train_test_split
# выполняет тасовку
kf = KFold(n_splits=SPLITS, shuffle=True, random_state=12345)

# Степени полинома
degrees = list(range(1, MAX_DEGREE+1))

best_degree = 0
best_mse = float("inf")

# Замечание: будем использовать один экземпляр и менять степень;
#  каждый раз модель будет заново обучаться; можно и каждый раз
#  создавать отдельный экземпляр
pipeline = Pipeline([
    ("transformation", PolynomialFeatures(degree=None, include_bias=False)), 
    ("linear_model", LinearRegression(fit_intercept=True))
])


# Выбор степени с наименьшей валидационной ошибкой 
for indx, degree in enumerate(degrees):

    pipeline.named_steps["transformation"].degree = degree
    scores = cross_validate(pipeline, X_train, y_train, cv=kf, return_train_score=True, scoring=["neg_mean_squared_error",])
    
    # Средняя ошибка на проверочном множестве
    mse_avg =  -scores["test_neg_mean_squared_error"].mean()
    
    if best_mse > mse_avg:
        best_mse = mse_avg
        best_degree = degree
    
    print("{}) Test MSE for degree {}: {}".format(indx+1, degree, mse_avg))

print("Best degree:", best_degree)

# Повторное обучение на всем обучающем множестве 
pipeline.named_steps["transformation"].degree = best_degree
pipeline.fit(X_train, y_train)

# Предсказания для тестового множества
y_test__pred = pipeline.predict(X_test)

# Ошибки на тестовом множестве
mse_test = mean_squared_error(y_test, y_test__pred)

print("Тестовое множество:")
print("\tTest MSE:", mse_test)

#### Вариант 2

In [ ]:
# Разбиение данных на обучающее и тестовое множества
X_train, X_test, y_train, y_test = train_test_split(X, y_true, test_size=0.3, random_state=200)

# Инициализация делителя для кросс-валидации
kf = KFold(n_splits=SPLITS, shuffle=True, random_state=12345)

# Степени полинома
degrees = list(range(1, MAX_DEGREE+1))

# Инициализация массива для MSE
scores = np.zeros((MAX_DEGREE, SPLITS))

pipeline = Pipeline([
    ("transformation", PolynomialFeatures(degree=None, include_bias=False)), 
    ("linear_model", LinearRegression(fit_intercept=True))
])

# Обучение
for i, (train_index, val_index) in enumerate(kf.split(X_train, y_train)):
    for j, degree in enumerate(degrees):
        pipeline.named_steps["transformation"].degree = degree
        pipeline.fit(X_train[train_index], y_train[train_index])
        scores[j, i] = mean_squared_error(y_train[val_index], pipeline.predict(X_train[val_index]))

# Средние проверочные ошибки для каждой степени
mses_avg = scores.mean(axis=1)

for indx, mse_avg in enumerate(mses_avg):
    print("{}) Test MSE for degree {}: {}".format(indx+1, degrees[indx], mse_avg))

indx_min__mse_avg = mses_avg.argmin()
best_degree = degrees[indx_min__mse_avg]

print("Best degree:", best_degree)

# Повторное обучение на всем обучающем множестве 
pipeline.named_steps["transformation"].degree = best_degree
pipeline.fit(X_train, y_train)

# Предсказания для тестового множества
y_test__pred = pipeline.predict(X_test)

# Ошибки на тестовом множестве
mse_test = mean_squared_error(y_test, y_test__pred)

print("Тестовое множество:")
print("\tTest MSE:", mse_test)

#### Вариант 3

In [ ]:
from joblib import Parallel, delayed

In [ ]:
def scores_on_split(train_index, val_index, max_degree=MAX_DEGREE):
    pipeline = Pipeline([
        ("transformation", PolynomialFeatures(degree=None, include_bias=False)), 
        ("linear_model", LinearRegression(fit_intercept=True))
    ])
    degrees = list(range(1, MAX_DEGREE+1))
    scores =np.zeros(max_degree)
    for j, degree in enumerate(degrees):
        pipeline.named_steps["transformation"].degree = degree
        pipeline.fit(X_train[train_index], y_train[train_index])
        scores[j] = mean_squared_error(y_train[val_index], pipeline.predict(X_train[val_index]))
    
    return scores

In [ ]:
# Разбиение данных на обучающее и тестовое множества
X_train, X_test, y_train, y_test = train_test_split(X, y_true, test_size=0.3, random_state=200)

# Инициализация делителя для кросс-валидации
kf = KFold(n_splits=SPLITS, shuffle=True, random_state=12345)

# Инициализация массива для MSE
scores = list()

# Параллельное обучение по сплитам
parallel = Parallel(n_jobs=4)
func = delayed(scores_on_split)
scores = np.asarray(parallel(func(train_index, val_index) for train_index, val_index in kf.split(X_train, y_train)))

# Средние проверочные ошибки для каждой степени
mses_avg = scores.mean(axis=0)

for indx, mse_avg in enumerate(mses_avg):
    print("{}) Test MSE for degree {}: {}".format(indx+1, degrees[indx], mse_avg))

indx_min__mse_avg = mses_avg.argmin()
best_degree = degrees[indx_min__mse_avg]

print("Best degree:", best_degree)

# Повторное обучение на всем обучающем множестве 
pipeline.named_steps["transformation"].degree = best_degree
pipeline.fit(X_train, y_train)

# Предсказания для тестового множества
y_test__pred = pipeline.predict(X_test)

# Ошибки на тестовом множестве
mse_test = mean_squared_error(y_test, y_test__pred)

print("Тестовое множество:")
print("\tTest MSE:", mse_test)

#### Вариант 4

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Разбиение данных на обучающее и тестовое множества
X_train, X_test, y_train, y_test = train_test_split(X, y_true, test_size=0.3, random_state=200)

# Инициализация делителя для кросс-валидации
kf = KFold(n_splits=SPLITS, shuffle=True, random_state=12345)

# Степени полинома
degrees = list(range(1, MAX_DEGREE+1))

# Инициализация массива для MSE
scores = np.zeros((MAX_DEGREE, SPLITS))

pipeline = Pipeline([
    ("transformation", PolynomialFeatures(degree=None, include_bias=False)), 
    ("linear_model", LinearRegression(fit_intercept=True))
])

# Сетка параметров
parameters = {
    "transformation__degree": degrees
}

# Параметры обучения
grid_class_parameters = {
    "estimator": pipeline,
    "param_grid": parameters,
    "cv": kf,
    "scoring": "neg_mean_squared_error"
}

# Обучение
grid_search = GridSearchCV(**grid_class_parameters)
grid_search.fit(X_train, y_train)

# Средние проверочные ошибки для каждой степени
mses_avg = np.abs(grid_search.cv_results_["mean_test_score"])

for indx, mse_avg in enumerate(mses_avg):
    print("{}) Test MSE for degree {}: {}".format(indx+1, degrees[indx], mse_avg))

print("Best degree:", grid_search.best_params_["transformation__degree"]  )

# Предсказания для тестового множества
y_test__pred = grid_search.predict(X_test)

# Ошибки на тестовом множестве
mse_test = mean_squared_error(y_test, y_test__pred)

print("Тестовое множество:")
print("\tTest MSE:", mse_test)

<a name="4"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:16pt; font-weight:bold">4. Источники</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a href="http://scikit-learn.org/stable/modules/cross_validation.html">3.1. Cross-validation: evaluating estimator performance</a>